In [3]:
!pip install PyMuPDF 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 2.4 MB/s eta 0:00:0000:0100:01


In [12]:
# 📂 Step 1: Import Libraries
import fitz  # PyMuPDF
import spacy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
import tkinter as tk
from tkinter import filedialog

# ✅ Load spaCy Model
nlp = spacy.load("en_core_web_sm")

In [14]:
# Module 1: Read and Print Resume PDF
import fitz  # PyMuPDF
from pathlib import Path

# Define the path to your resume PDF
pdf_path = "TE_Resume.pdf"  # <- Change this to your actual PDF path

# Open the PDF
doc = fitz.open(pdf_path)

# Extract and print text from all pages
for page_num, page in enumerate(doc, start=1):
    resume_text = page.get_text()
    text = resume_text
    print(f"\n--- Page {page_num} ---\n")
    print(resume_text)


--- Page 1 ---

NIKHIL .M. SUTAR
SUMMARY
Innovative developer with expertise in C, Java, Python, and JavaScript. Proficient in utilizing frameworks such
as Flask, Django, and Bootstrap, with ongoing learning in React.js. Experienced in version control with Git and
GitHub and adept at deploying projects on Vercel. Committed to leveraging AI tools, including ChatGPT, to
enhance project outcomes. Seeking a dynamic role to apply technical skills and drive project success.
PROJECTS
EDUCATION
Live Link : https://success-classes-of-engineering.vercel.app/
Frontend : HTML , CSS 
Backend : Flask , Python
2. Success Classes of Engineering
    Jun 2024  - Jun 2024
ADDITIONAL INFORMATION
Another Skills Certification :  PC Operation (Excel, PowerPoint, Word), Power BI
Languages: English, Hindi , Marathi
Hobbies: Drawing Architectural structures 
4. Portfolio Website
Live Link : https://carpentrycoder.github.io/Resume/
Frontend: HTML , JS , Tailwind CSS
Jan 2025 - Jan 2025
Github | +91-7738544966 |

In [34]:
import re
# 🧠 Step 3: NLP Processing to Detect Skills
doc = nlp(resume_text)

# Custom keywords (can be expanded)
keywords = [
    'Python', 'Java', 'C', 'JavaScript', 'Django', 'Flask', 'React', 'Machine Learning',
    'LLM', 'SQL', 'Pandas', 'Numpy', 'Seaborn', 'Bootstrap', 'GitHub', 'Tailwind',
    'C++', 'TypeScript', 'HTML', 'CSS', 'Tailwind CSS', 'React.js', 'Vue.js', 'Angular',
    'Node.js', 'Express.js', 'NumPy', 'Matplotlib', 'Plotly', 'Scikit-Learn', 'Deep Learning',
    'LLMs', 'Hugging Face', 'Transformers', 'TensorFlow', 'PyTorch', 'OpenCV', 'Power BI',
    'Excel', 'Git', 'Docker', 'PostgreSQL', 'MongoDB'
]

def extract_email(text):
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group(0) if match else None

def extract_phone(text):
    match = re.search(r'\+91[-\s]?[0-9]{10}|\b[7-9]\d{9}\b', text)
    return match.group(0) if match else None

def extract_name(text):
    lines = text.split("\n")
    for line in lines:
        if "SUMMARY" in line.upper():
            return lines[lines.index(line) - 1].strip()
    return "Name not found"

def extract_education(text):
    education_keywords = ['university', 'college', 'bachelor', 'master', 'b.tech', 'm.tech', 'cgpa', 'degree']
    lines = text.lower().split("\n")
    education = [line.strip() for line in lines if any(keyword in line for keyword in education_keywords)]
    return education if education else ["Education info not found"]

email = extract_email(resume_text)
phone = extract_phone(resume_text)
name = extract_name(resume_text)
education = extract_education(resume_text)

# 🖨️ Output Results
print("Name:", name)
print("\nEmail:", email)
print("\nPhone:", phone)
print("\neducation:", education)



detected_skills = [word for word in keywords if word.lower() in resume_text.lower()]
print("\n✅ Detected Skills:\n", detected_skills)

# ✅ Store all variables in resume_data dictionary
resume_data = {
    "name": name,
    "email": email,
    "phone": phone,
    "education": education,
    "skills": detected_skills,
    "skills_count": len(detected_skills),
}
print(f"\n{resume_data}")


Name: NIKHIL .M. SUTAR

Email: nikhilsutar621@gmail.com

Phone: +91-7738544966

education: ['konkan gyanpeeth college of engineering , affiliated to the university of mumbai', 'artificial intelligence and data science | cgpa of 5 sems: 6.70']

✅ Detected Skills:
 ['Python', 'Java', 'C', 'JavaScript', 'Django', 'Flask', 'React', 'Machine Learning', 'LLM', 'SQL', 'Pandas', 'Numpy', 'Seaborn', 'Bootstrap', 'GitHub', 'Tailwind', 'HTML', 'CSS', 'Tailwind CSS', 'React.js', 'Node.js', 'NumPy', 'Matplotlib', 'Scikit-Learn', 'LLMs', 'Hugging Face', 'Transformers', 'Power BI', 'Excel', 'Git', 'PostgreSQL']

{'name': 'NIKHIL .M. SUTAR', 'email': 'nikhilsutar621@gmail.com', 'phone': '+91-7738544966', 'education': ['konkan gyanpeeth college of engineering , affiliated to the university of mumbai', 'artificial intelligence and data science | cgpa of 5 sems: 6.70'], 'skills': ['Python', 'Java', 'C', 'JavaScript', 'Django', 'Flask', 'React', 'Machine Learning', 'LLM', 'SQL', 'Pandas', 'Numpy', 'Seabor

In [10]:
# 🎯 Step 4: Career Role Matching
roles = {
    "Machine Learning Engineer": ["Python", "Scikit-Learn", "Pandas", "Numpy", "ML", "Hugging Face", "LLM"],
    "Full Stack Developer": ["HTML", "CSS", "JavaScript", "React", "Django", "Flask", "SQL"],
    "AI Research Assistant": ["Python", "Transformers", "LLMs", "Deep Learning", "NLP"],
}

def match_score(user_skills, role_skills):
    return len(set(user_skills).intersection(set(role_skills))) / len(role_skills)

suggestions = []
for role, skills_required in roles.items():
    score = match_score(detected_skills, skills_required)
    suggestions.append((role, round(score * 100, 2)))

career_df = pd.DataFrame(suggestions, columns=["Career Role", "Match %"]).sort_values("Match %", ascending=False)

print("\n🔮 Top Career Suggestions:")
display(career_df)


🔮 Top Career Suggestions:


Career Role  Match %
1       Full Stack Developer   100.00
0  Machine Learning Engineer    85.71
2      AI Research Assistant    60.00

In [11]:
# 🧩 Step 5: Skill Gap Analysis
def skill_gap(role, user_skills):
    required = set(roles[role])
    missing = required - set(user_skills)
    return list(missing)

selected_role = career_df.iloc[0]["Career Role"]
gaps = skill_gap(selected_role, detected_skills)

print(f"\n📌 Skill Gaps for '{selected_role}':")
print(gaps)


📌 Skill Gaps for 'Full Stack Developer':
[]


Final Def 

In [16]:
import re
# 🧠 Step 3: NLP Processing to Detect Skills
doc = nlp(resume_text)

# Custom keywords (can be expanded)
keywords = [
    'Python', 'Java', 'C', 'JavaScript', 'Django', 'Flask', 'React', 'Machine Learning',
    'LLM', 'SQL', 'Pandas', 'Numpy', 'Seaborn', 'Bootstrap', 'GitHub', 'Tailwind',
    'C++', 'TypeScript', 'HTML', 'CSS', 'Tailwind CSS', 'React.js', 'Vue.js', 'Angular',
    'Node.js', 'Express.js', 'NumPy', 'Matplotlib', 'Plotly', 'Scikit-Learn', 'Deep Learning',
    'LLMs', 'Hugging Face', 'Transformers', 'TensorFlow', 'PyTorch', 'OpenCV', 'Power BI',
    'Excel', 'Git', 'Docker', 'PostgreSQL', 'MongoDB'
]

def extract_email(text):
    match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    return match.group(0) if match else None

def extract_phone(text):
    match = re.search(r'\+91[-\s]?[0-9]{10}|\b[7-9]\d{9}\b', text)
    return match.group(0) if match else None

def extract_name(text):
    lines = text.split("\n")
    for line in lines:
        if "SUMMARY" in line.upper():
            return lines[lines.index(line) - 1].strip()
    return "Name not found"

def extract_education(text):
    education_keywords = ['university', 'college', 'bachelor', 'master', 'b.tech', 'm.tech', 'cgpa', 'degree']
    lines = text.lower().split("\n")
    education = [line.strip() for line in lines if any(keyword in line for keyword in education_keywords)]
    return education if education else ["Education info not found"]

email = extract_email(resume_text)
phone = extract_phone(resume_text)
name = extract_name(resume_text)
education = extract_education(resume_text)

# 🖨️ Output Results
print("Name:", name)
print("\nEmail:", email)
print("\nPhone:", phone)
print("\neducation:", education)



detected_skills = [word for word in keywords if word.lower() in resume_text.lower()]
print("\n✅ Detected Skills:\n", detected_skills)

# 🎯 Step 4: Career Role Matching
roles = {
    "Machine Learning Engineer": ["Python", "Scikit-Learn", "Pandas", "Numpy", "ML", "Hugging Face", "LLM"],
    "Full Stack Developer": ["HTML", "CSS", "JavaScript", "React", "Django", "Flask", "SQL"],
    "AI Research Assistant": ["Python", "Transformers", "LLMs", "Deep Learning", "NLP"],
}

def match_score(user_skills, role_skills):
    return len(set(user_skills).intersection(set(role_skills))) / len(role_skills)

suggestions = []
for role, skills_required in roles.items():
    score = match_score(detected_skills, skills_required)
    suggestions.append((role, round(score * 100, 2)))

career_df = pd.DataFrame(suggestions, columns=["Career Role", "Match %"]).sort_values("Match %", ascending=False)

print("\n🔮 Top Career Suggestions:")
display(career_df)

# 🧩 Step 5: Skill Gap Analysis
def skill_gap(role, user_skills):
    required = set(roles[role])
    missing = required - set(user_skills)
    return list(missing)

selected_role = career_df.iloc[0]["Career Role"]
gaps = skill_gap(selected_role, detected_skills)

print(f"\n📌 Skill Gaps for '{selected_role}':")
print(gaps)

Name: NIKHIL .M. SUTAR

Email: nikhilsutar621@gmail.com

Phone: +91-7738544966

education: ['konkan gyanpeeth college of engineering , affiliated to the university of mumbai', 'artificial intelligence and data science | cgpa of 5 sems: 6.70']

✅ Detected Skills:
 ['Python', 'Java', 'C', 'JavaScript', 'Django', 'Flask', 'React', 'Machine Learning', 'LLM', 'SQL', 'Pandas', 'Numpy', 'Seaborn', 'Bootstrap', 'GitHub', 'Tailwind', 'HTML', 'CSS', 'Tailwind CSS', 'React.js', 'Node.js', 'NumPy', 'Matplotlib', 'Scikit-Learn', 'LLMs', 'Hugging Face', 'Transformers', 'Power BI', 'Excel', 'Git', 'PostgreSQL']

🔮 Top Career Suggestions:


Career Role  Match %
1       Full Stack Developer   100.00
0  Machine Learning Engineer    85.71
2      AI Research Assistant    60.00


📌 Skill Gaps for 'Full Stack Developer':
[]


In [18]:
!pip install pandas numpy scikit-learn xgboost -q

In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from xgboost import XGBClassifier